# Construcción de Operaciones

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [109]:
import pandas as pd
from abc import ABC, abstractmethod
from pydantic import BaseModel, NonNegativeInt
from typing import Optional, Any, Union

import qcfinancial as qcf

# Local module
import aux_functions as aux
import qcf_wrappers as qcw

Se verifica la versión y build de `qcfinancial`.

In [2]:
qcf.id()

'version: save-present-value, build: 2024-06-19 15:10'

El siguiente diccionario se utiliza para dar formato a las columnas de los `pandas.DataFrames` que se utilizarán.

In [3]:
format_dict = {
    'nominal': '{0:,.2f}', 
    'nocional': '{0:,.2f}', 
    'amort': '{0:,.2f}', 
    'amortizacion': '{0:,.2f}', 
    'interes': '{0:,.2f}',
    'monto': '{0:,.2f}',
    'flujo': '{0:,.2f}',
    'flujo_moneda_pago': '{0:,.2f}',
    'flujo_en_clp': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}', 
    'icp_final': '{0:,.2f}',
    'uf_inicial': '{0:,.2f}', 
    'uf_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}', 
    'spread': '{0:,.4%}', 
    'gearing': '{0:,.2f}',
    'amortizacion_moneda_pago': '{0:,.2f}', 
    'interes_moneda_pago': '{0:,.2f}', 
    'valor_indice_fx': '{0:,.2f}'
}

In [4]:
all_calendars = {
    "SANTIAGO": qcf.BusinessCalendar(qcf.QCDate(1, 1, 2024), 20),
}

Se da de alta la UF y el dólar observado.

In [5]:
clf = qcf.QCCLF()
clp = qcf.QCCLP()
usd = qcf.QCUSD()
usdclp = qcf.FXRate(usd, clp)
clfclp = qcf.FXRate(clf, clp)
zero_d = qcf.Tenor('0D')
one_d = qcf.Tenor('1D')

In [6]:
all_fx_rate_indices = {
    "UF": qcf.FXRateIndex(
        clfclp, 
        'UF', 
        zero_d, 
        zero_d, 
        all_calendars["SANTIAGO"],
    ),
    "USDCLP_OBS": qcf.FXRateIndex(
        usdclp, 
        'USDCLP_OBS', 
        one_d, 
        one_d, 
        all_calendars["SANTIAGO"],
    )
}

### Interfaz

In [115]:
class Seed(ABC):
    @abstractmethod
    def get_parameters(self, *args, **kwargs) -> dict[str, Any]:
        pass
    
class QcfLegGenerator(ABC):
    @abstractmethod
    def get_qcf_leg(self, *args, **kwargs) -> qcf.Leg:
        pass

## Construcción Asistida de un `FixedRateLeg`

Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.
- `SettLagBehaviour`: este parámetro indica cómo se calcula un `settlement_date` cuando un `end_date` cae en un día festivo. Las alternativas son dos, se calcula sumando el `settlement_lag` desde `end_date` o se calcula sumando `settlement_lag` a partir de la primera fecha hábil posterior a `end_date`.

El parámetro `SettLagBehaviour` se agregó en la versión 0.11.0 .

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

Se da de alta los parámetros requeridos

In [44]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(5, 11, 2023)
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
calendario = all_calendars['SANTIAGO']
lag_pago = 1
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False
sett_lag_behaviour = qcf.SettLagBehaviour.DONT_MOVE

Se da de alta el objeto.

In [45]:
fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono,
    sett_lag_behaviour,
)

Se muestra el resultado.

In [46]:
aux.leg_as_dataframe(fixed_rate_leg)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-05,2020-05-06,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360
1,2020-05-05,2020-11-05,2020-11-06,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
2,2020-11-05,2021-05-05,2021-05-06,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
3,2021-05-05,2021-11-05,2021-11-08,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
4,2021-11-05,2022-05-05,2022-05-06,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
5,2022-05-05,2022-11-05,2022-11-07,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
6,2022-11-05,2023-05-05,2023-05-08,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
7,2023-05-05,2023-11-05,2023-11-06,100000.0,100000.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360


### Templates

In [49]:
class FixedRateLegSeed(BaseModel, Seed):
    rec_pay: qcw.AP
    start_date: qcw.Fecha
    maturity: qcw.Tenor
    initial_notional: float
    interest_rate_value: float
    
    def get_parameters(self, type_of_rate: qcw.TypeOfRate) -> dict[str, Any]:
        qcf_start_date = self.start_date.as_qcf()
        months = self.maturity.total_months()
        return {
            "rec_pay": self.rec_pay.as_qcf(),
            "start_date": qcf_start_date,
            "end_date": qcf_start_date.add_months(months),
            "initial_notional": self.initial_notional,
            "interest_rate": type_of_rate.as_qcf_with_value(self.interest_rate_value),
        }

In [54]:
class FixedRateLegTemplate(BaseModel, Seed, QcfLegGenerator):
    bus_adj_rule: qcw.BusAdjRules
    settlement_periodicity: qcw.Tenor
    settlement_stub_period: qcw.StubPeriods
    settlement_calendar: str
    settlement_lag: NonNegativeInt
    amort_is_cashflow: bool
    interest_rate: qcw.TypeOfRate
    notional_currency: qcw.Currency
    is_bond: bool
    sett_lag_behaviour: qcw.SettLagBehaviour
    
    def get_parameters(self, seed: FixedRateLegSeed, calendars: dict[str, qcf.BusinessCalendar]):
        seed_parameters = seed.get_parameters(self.interest_rate)
        template_parameters = {
            "bus_adj_rule": self.bus_adj_rule.as_qcf(),
            "initial_notional": seed.initial_notional,
            "settlement_periodicity": self.settlement_periodicity.as_qcf(),
            "settlement_stub_period": self.settlement_stub_period.as_qcf(),
            "settlement_calendar": calendars[self.settlement_calendar],
            "settlement_lag": self.settlement_lag,
            "amort_is_cashflow": self.amort_is_cashflow,
            "interest_rate": self.interest_rate.as_qcf_with_value(seed.interest_rate_value),
            "notional_currency": self.notional_currency.as_qcf(),
            "is_bond": self.is_bond,
            "sett_lag_behaviour": self.sett_lag_behaviour.as_qcf(),
        }
        return seed_parameters | template_parameters
    
    def get_qcf_leg(self, seed: FixedRateLegSeed, calendars: dict[str, qcf.BusinessCalendar]) -> qcf.Leg:
        return qcf.LegFactory.build_bullet_fixed_rate_leg(
            **self.get_parameters(seed, calendars),
        )

In [55]:
pata_fija_template = FixedRateLegTemplate(
    bus_adj_rule=qcw.BusAdjRules.MOD_FOLLOW,
    settlement_periodicity=qcw.Tenor(dias=0, meses=6, agnos=0),
    settlement_stub_period=qcw.StubPeriods.NO,
    settlement_calendar="SANTIAGO",
    settlement_lag=1,
    amort_is_cashflow=True,
    interest_rate=qcw.TypeOfRate.LINACT360,
    notional_currency=qcw.Currency.CLP,
    is_bond=False,
    sett_lag_behaviour=qcw.SettLagBehaviour.DONT_MOVE,
)

In [56]:
pata_fija_seed = FixedRateLegSeed(
    rec_pay=qcw.AP.A,
    start_date=qcw.Fecha(fecha="2024-06-28"),
    maturity=qcw.Tenor(dias=0, meses=0, agnos=2),
    initial_notional=100_000_000,
    interest_rate_value=.05,
)

In [57]:
aux.leg_as_dataframe(pata_fija_template.get_qcf_leg(pata_fija_seed, all_calendars))

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2024-06-28,2024-12-30,2024-12-31,100000000.0,0.0,2.569444e+06,True,2.569444e+06,CLP,0.05,LinAct360
1,2024-12-30,2025-06-30,2025-07-01,100000000.0,0.0,2.527778e+06,True,2.527778e+06,CLP,0.05,LinAct360
2,2025-06-30,2025-12-29,2025-12-30,100000000.0,0.0,2.527778e+06,True,2.527778e+06,CLP,0.05,LinAct360
3,2025-12-29,2026-06-29,2026-06-30,100000000.0,100000000.0,2.527778e+06,True,1.025278e+08,CLP,0.05,LinAct360


## Construcción Asistida de un `FixedRateMultiCurrencyLeg`

Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateMultiCurrencyCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateMultiCurrencyCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago.
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.
- `SettLagBehaviour`: este parámetro indica cómo se calcula un `settlement_date` cuando un `end_date` cae en un día festivo.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

Luego se dan de alta los otros parámetros requeridos para la construcción

In [58]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(12, 7, 1968)
fecha_final = qcf.QCDate(12, 7, 1974) 
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
lag_pago = 1
es_bono = False
sett_lag_behaviour = qcf.SettLagBehaviour.DONT_MOVE

Finalmente, se da de alta el objeto.

In [59]:
fixed_rate_mccy_leg = qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    clf,
    clp,
    all_fx_rate_indices["UF"],
    0,
    es_bono,
    sett_lag_behaviour,
)

Visualización.

In [60]:
aux.leg_as_dataframe(fixed_rate_mccy_leg)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda_nocional,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,indice_fx,valor_indice_fx,amortizacion_moneda_pago,interes_moneda_pago
0,1968-07-12,1969-01-12,1969-01-13,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360,1969-01-13,CLP,UF,1.0,0.0,1533.333333
1,1969-01-12,1969-07-12,1969-07-14,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360,1969-07-14,CLP,UF,1.0,0.0,1508.333333
2,1969-07-12,1970-01-12,1970-01-13,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360,1970-01-13,CLP,UF,1.0,0.0,1533.333333
3,1970-01-12,1970-07-12,1970-07-13,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360,1970-07-13,CLP,UF,1.0,0.0,1508.333333
4,1970-07-12,1971-01-12,1971-01-13,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360,1971-01-13,CLP,UF,1.0,0.0,1533.333333
5,1971-01-12,1971-07-12,1971-07-13,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360,1971-07-13,CLP,UF,1.0,0.0,1508.333333
6,1971-07-12,1972-01-12,1972-01-13,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360,1972-01-13,CLP,UF,1.0,0.0,1533.333333
7,1972-01-12,1972-07-12,1972-07-13,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360,1972-07-13,CLP,UF,1.0,0.0,1516.666667
8,1972-07-12,1973-01-12,1973-01-15,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360,1973-01-15,CLP,UF,1.0,0.0,1533.333333
9,1973-01-12,1973-07-12,1973-07-13,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360,1973-07-13,CLP,UF,1.0,0.0,1508.333333


### Template

In [61]:
class MultiCurrencySeed(BaseModel, Seed):
    settlement_currency: qcw.Currency
    fx_rate_index_name: str
    fx_rate_index_fixing_lag: NonNegativeInt
    
    def get_parameters(self, indices: dict[str, qcf.FXRateIndex]):
        return {
            "settlement_currency": self.settlement_currency.as_qcf(),
            "fx_rate_index": indices[self.fx_rate_index_name],
            "fx_rate_index_fixing_lag": self.fx_rate_index_fixing_lag,
        }

In [69]:
class FixedRateMultiCurrencyLegTemplate(BaseModel, Seed, QcfLegGenerator):
    fixed_rate_template: FixedRateLegTemplate
    multi_currency_template: MultiCurrencySeed
    
    def get_parameters(
            self,
            seed: FixedRateLegSeed, 
            calendars: dict[str, qcf.BusinessCalendar],
            indices: dict[str, qcf.FXRateIndex],
    ) -> dict[str, Any]:
        fixed_rate_leg_parameters = self.fixed_rate_template.get_parameters(seed, calendars)
        mmcy_parameters = {
            "settlement_currency": self.multi_currency_template.settlement_currency.as_qcf(),
            "fx_rate_index": indices[self.multi_currency_template.fx_rate_index_name],
            "fx_rate_index_fixing_lag": self.multi_currency_template.fx_rate_index_fixing_lag,
        }
        return fixed_rate_leg_parameters | mmcy_parameters
    
    def get_qcf_leg(
            self, 
            seed: FixedRateLegSeed, 
            calendars: dict[str, qcf.BusinessCalendar],
            indices: dict[str, qcf.FXRateIndex],
    ) -> qcf.Leg:
        return qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(**self.get_parameters(seed, calendars, indices))

In [70]:
pata_fija_uf_template = FixedRateLegTemplate(
    bus_adj_rule=qcw.BusAdjRules.MOD_FOLLOW,
    settlement_periodicity=qcw.Tenor(dias=0, meses=6, agnos=0),
    settlement_stub_period=qcw.StubPeriods.NO,
    settlement_calendar="SANTIAGO",
    settlement_lag=1,
    amort_is_cashflow=True,
    interest_rate=qcw.TypeOfRate.LINACT360,
    notional_currency=qcw.Currency.CLF,
    is_bond=False,
    sett_lag_behaviour=qcw.SettLagBehaviour.DONT_MOVE,
)

mccy_uf_template = MultiCurrencySeed(
    settlement_currency=qcw.Currency.CLP,
    fx_rate_index_name="UF",
    fx_rate_index_fixing_lag=0,
)

pata_fija_mccy_template = FixedRateMultiCurrencyLegTemplate(
    fixed_rate_template=pata_fija_uf_template,
    multi_currency_template=mccy_uf_template,
)

pata_fija_mccy_seed = FixedRateLegSeed(
    rec_pay=qcw.AP.A,
    start_date=qcw.Fecha(fecha="2024-06-28"),
    maturity=qcw.Tenor(dias=0, meses=0, agnos=2),
    initial_notional=300_000,
    interest_rate_value=.02,
)

In [71]:
aux.leg_as_dataframe(pata_fija_mccy_template.get_qcf_leg(pata_fija_seed, all_calendars, all_fx_rate_indices))

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda_nocional,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,indice_fx,valor_indice_fx,amortizacion_moneda_pago,interes_moneda_pago
0,2024-06-28,2024-12-30,2024-12-31,100000000.0,0.0,2.569444e+06,True,2.569444e+06,CLF,0.05,LinAct360,2024-12-31,CLP,UF,1.0,0.0,2.569444e+06
1,2024-12-30,2025-06-30,2025-07-01,100000000.0,0.0,2.527778e+06,True,2.527778e+06,CLF,0.05,LinAct360,2025-07-01,CLP,UF,1.0,0.0,2.527778e+06
2,2025-06-30,2025-12-29,2025-12-30,100000000.0,0.0,2.527778e+06,True,2.527778e+06,CLF,0.05,LinAct360,2025-12-30,CLP,UF,1.0,0.0,2.527778e+06
3,2025-12-29,2026-06-29,2026-06-30,100000000.0,100000000.0,2.527778e+06,True,1.025278e+08,CLF,0.05,LinAct360,2026-06-30,CLP,UF,1.0,100000000.0,2.527778e+06


## Construcción Asistida de un `OvernightIndexLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: tipo de ajuste en la fecha de fijación de los valores inicial y final del índice en cada cupón
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación del índice
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales de la tasa equivalente
- `QCCurrency`: moneda del nocional
- `DatesForEquivalentRate`: enum que indica qué fechas se utilizan en el cálculo de la tasa equivalente (fechas de devengo o de índice)
- `SettLagBehaviour`: este parámetro indica cómo se calcula un `settlement_date` cuando un `end_date` cae en un día festivo.

**NOTA:** para construir un `Leg` con `OvernightIndexCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_overnight_index_leg(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [72]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2029) 
bus_adj_rule = qcf.BusyAdjRules.NO
index_adj_rule = qcf.BusyAdjRules.FOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
num_decimales_tasa_eq = 8
lag_pago = 0
nominal = 100_000_000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0
nombre_indice = 'ICPCLP'
sett_lag_behaviour = qcf.SettLagBehaviour.MOVE_TO_WORKING_DAY

Finalmente, se da de alta el objeto.

In [73]:
on_index_leg = qcf.LegFactory.build_bullet_overnight_index_leg(
    rp, 
    fecha_inicio,
    fecha_final, 
    bus_adj_rule, 
    index_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    calendario,
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    nombre_indice,
    num_decimales_tasa_eq,
    clp,
    qcf.DatesForEquivalentRate.ACCRUAL,
    sett_lag_behaviour,
)

Se visualiza.

In [74]:
aux.leg_as_dataframe(on_index_leg)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,2024-01-31,2024-07-31,2024-01-31,2024-07-31,2024-07-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,505555.555556,5.055556e+05,0.01,1.0
1,2024-07-31,2025-01-31,2024-07-31,2025-01-31,2025-01-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,511111.111111,5.111111e+05,0.01,1.0
2,2025-01-31,2025-07-31,2025-01-31,2025-07-31,2025-07-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,502777.777778,5.027778e+05,0.01,1.0
3,2025-07-31,2026-01-31,2025-07-31,2026-02-02,2026-02-02,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,511111.111111,5.111111e+05,0.01,1.0
4,2026-01-31,2026-07-31,2026-02-02,2026-07-31,2026-07-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,502777.777778,5.027778e+05,0.01,1.0
5,2026-07-31,2027-01-31,2026-07-31,2027-02-01,2027-02-01,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,511111.111111,5.111111e+05,0.01,1.0
6,2027-01-31,2027-07-31,2027-02-01,2027-08-02,2027-08-02,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,502777.777778,5.027778e+05,0.01,1.0
7,2027-07-31,2028-01-31,2027-08-02,2028-01-31,2028-01-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,511111.111111,5.111111e+05,0.01,1.0
8,2028-01-31,2028-07-31,2028-01-31,2028-07-31,2028-07-31,100000000.0,0.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,505555.555556,5.055556e+05,0.01,1.0
9,2028-07-31,2029-01-31,2028-07-31,2029-01-31,2029-01-31,100000000.0,100000000.0,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,511111.111111,1.005111e+08,0.01,1.0


### Templates

In [75]:
class OvernightIndexLegSeed(BaseModel, Seed):
    rec_pay: qcw.AP
    start_date: qcw.Fecha
    maturity: qcw.Tenor
    initial_notional: float
    spread: float
    gearing: Optional[float] = 1.0
    
    def get_parameters(self) -> dict[str, Any]:
        qcf_start_date = self.start_date.as_qcf()
        months = self.maturity.total_months()
        return {
            "rec_pay": self.rec_pay.as_qcf(),
            "start_date": qcf_start_date,
            "end_date": qcf_start_date.add_months(months),
            "initial_notional": self.initial_notional,
            "spread": self.spread,
            "gearing": self.gearing,
        }
    
class OvernightIndexLegTemplate(BaseModel, Seed, QcfLegGenerator):
    bus_adj_rule: qcw.BusAdjRules
    fix_adj_rule: qcw.BusAdjRules
    settlement_periodicity: qcw.Tenor
    stub_period: qcw.StubPeriods
    settlement_calendar: str
    fixing_calendar: str
    settlement_lag: NonNegativeInt
    amort_is_cashflow: bool
    interest_rate: qcw.TypeOfRate
    index_name: str
    eq_rate_decimal_places: NonNegativeInt
    notional_currency: qcw.Currency
    dates_for_eq_rate: qcw.DatesForEquivalentRate
    sett_lag_behaviour: qcw.SettLagBehaviour
    
    def get_parameters(
            self, 
            calendars: dict[str, qcf.BusinessCalendar]
    ) -> dict[str, Any]:
        qcf_sett_cal = calendars[self.settlement_calendar]
        qcf_fix_cal = calendars[self.fixing_calendar]
        return {
            "bus_adj_rule": self.bus_adj_rule.as_qcf(),
            "fix_adj_rule": self.fix_adj_rule.as_qcf(),
            "settlement_periodicity": self.settlement_periodicity.as_qcf(),
            "stub_period": self.stub_period.as_qcf(),
            "settlement_calendar": qcf_sett_cal,
            "fixing_calendar": qcf_fix_cal,
            "settlement_lag": self.settlement_lag,
            "amort_is_cashflow": self.amort_is_cashflow,
            "interest_rate":self.interest_rate.as_qcf(),
            "index_name": self.index_name,
            "eq_rate_decimal_places": self.eq_rate_decimal_places,
            "notional_currency": self.notional_currency.as_qcf(),
            "dates_for_eq_rate": self.dates_for_eq_rate.as_qcf(),
            "sett_lag_behaviour": self.sett_lag_behaviour.as_qcf(),
        }
    
    def get_qcf_leg(self, seed: OvernightIndexLegSeed, calendars: dict[str, qcf.BusinessCalendar]) -> qcf.Leg:
        seed_parameters = seed.get_parameters()
        template_parameters = self.get_parameters(calendars)
        return qcf.LegFactory.build_bullet_overnight_index_leg(**(seed_parameters | template_parameters))

In [76]:
overnight_index_leg_seed = OvernightIndexLegSeed(
    rec_pay=qcw.AP.P,
    start_date=qcw.Fecha(fecha="2024-06-28"),
    maturity=qcw.Tenor(dias=0, meses=0, agnos=2),
    initial_notional=1_000_000_000,
    spread=0.01,
)

In [77]:
overnight_index_leg_template = OvernightIndexLegTemplate(
    bus_adj_rule=qcw.BusAdjRules.MOD_FOLLOW,
    fix_adj_rule=qcw.BusAdjRules.FOLLOW,
    settlement_periodicity=qcw.Tenor(dias=0, meses=6, agnos=0),
    stub_period=qcw.StubPeriods.NO,
    settlement_calendar="SANTIAGO",
    fixing_calendar="SANTIAGO",
    settlement_lag=1,
    amort_is_cashflow=True,
    interest_rate=qcw.TypeOfRate.LINACT360,
    index_name="ICPCLP",
    eq_rate_decimal_places=6,
    notional_currency=qcw.Currency.CLP,
    dates_for_eq_rate=qcw.DatesForEquivalentRate.ACCRUAL,
    sett_lag_behaviour=qcw.SettLagBehaviour.DONT_MOVE,
)

In [79]:
aux.leg_as_dataframe(overnight_index_leg_template.get_qcf_leg(overnight_index_leg_seed, all_calendars))

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,2024-06-28,2024-12-30,2024-06-28,2024-12-30,2024-12-31,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,-5.138889e+06,-5.138889e+06,0.01,1.0
1,2024-12-30,2025-06-30,2024-12-30,2025-06-30,2025-07-01,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,-5.055556e+06,-5.055556e+06,0.01,1.0
2,2025-06-30,2025-12-29,2025-06-30,2025-12-29,2025-12-30,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,-5.055556e+06,-5.055556e+06,0.01,1.0
3,2025-12-29,2026-06-29,2025-12-29,2026-06-29,2026-06-30,-1.000000e+09,-1.000000e+09,True,CLP,ICPCLP,1.0,1.0,0.0,LinAct360,-5.055556e+06,-1.005056e+09,0.01,1.0


## Construcción Asistida de un `OvernightIndexMultiCurrencyLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: tipo de ajuste en la fecha de fijación de los valores inicial y final del índice en cada cupón
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación del índice
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales de la tasa equivalente
- `QCCurrency`: moneda del nocional
- `DatesForEquivalentRate`: enum que indica qué fechas se utilizan en el cálculo de la tasa equivalente (fechas de devengo o de índice)
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)
- `SettLagBehaviour`: este parámetro indica cómo se calcula un `settlement_date` cuando un `end_date` cae en un día festivo.

**NOTA:** para construir un `Leg` con `OvernightIndexMultiCurrencyCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_overnight_index_multi_currency_leg(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [80]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2029) 
bus_adj_rule = qcf.BusyAdjRules.NO
index_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
num_decimales_tasa_eq = 8
lag_pago = 0
nominal = 100_000_000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0
nombre_indice = 'ICPCLP'
sett_lag_behaviour = qcf.SettLagBehaviour.MOVE_TO_WORKING_DAY

Finalmente, se da de alta el objeto.

In [81]:
on_index_mccy_leg = qcf.LegFactory.build_bullet_overnight_index_multi_currency_leg(
    rp, 
    fecha_inicio,
    fecha_final, 
    bus_adj_rule, 
    index_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    calendario,
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    nombre_indice,
    num_decimales_tasa_eq,
    clp,
    qcf.DatesForEquivalentRate.ACCRUAL,
    usd,
    all_fx_rate_indices["USDCLP_OBS"],
    0,
    sett_lag_behaviour,
)

Se visualiza.

In [82]:
aux.leg_as_dataframe(on_index_mccy_leg)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2024-01-31,2024-07-31,2024-01-31,2024-07-31,2024-07-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.055556e+05,0.01,1.0,USD,USDCLP_OBS,2024-07-31,1.0,505556.0,0.0,505556.0
1,2024-07-31,2025-01-31,2024-07-31,2025-01-31,2025-01-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.111111e+05,0.01,1.0,USD,USDCLP_OBS,2025-01-31,1.0,511111.0,0.0,511111.0
2,2025-01-31,2025-07-31,2025-01-31,2025-07-31,2025-07-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.027778e+05,0.01,1.0,USD,USDCLP_OBS,2025-07-31,1.0,502778.0,0.0,502778.0
3,2025-07-31,2026-01-31,2025-07-31,2026-01-30,2026-02-02,100000000.0,0.0,True,CLP,ICPCLP,...,5.111111e+05,0.01,1.0,USD,USDCLP_OBS,2026-02-02,1.0,511111.0,0.0,511111.0
4,2026-01-31,2026-07-31,2026-01-30,2026-07-31,2026-07-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.027778e+05,0.01,1.0,USD,USDCLP_OBS,2026-07-31,1.0,502778.0,0.0,502778.0
5,2026-07-31,2027-01-31,2026-07-31,2027-01-29,2027-02-01,100000000.0,0.0,True,CLP,ICPCLP,...,5.111111e+05,0.01,1.0,USD,USDCLP_OBS,2027-02-01,1.0,511111.0,0.0,511111.0
6,2027-01-31,2027-07-31,2027-01-29,2027-07-30,2027-08-02,100000000.0,0.0,True,CLP,ICPCLP,...,5.027778e+05,0.01,1.0,USD,USDCLP_OBS,2027-08-02,1.0,502778.0,0.0,502778.0
7,2027-07-31,2028-01-31,2027-07-30,2028-01-31,2028-01-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.111111e+05,0.01,1.0,USD,USDCLP_OBS,2028-01-31,1.0,511111.0,0.0,511111.0
8,2028-01-31,2028-07-31,2028-01-31,2028-07-31,2028-07-31,100000000.0,0.0,True,CLP,ICPCLP,...,5.055556e+05,0.01,1.0,USD,USDCLP_OBS,2028-07-31,1.0,505556.0,0.0,505556.0
9,2028-07-31,2029-01-31,2028-07-31,2029-01-31,2029-01-31,100000000.0,100000000.0,True,CLP,ICPCLP,...,1.005111e+08,0.01,1.0,USD,USDCLP_OBS,2029-01-31,1.0,511111.0,100000000.0,100511111.0


### Templates

In [83]:
class OvernightIndexMultiCurrencyLegTemplate(BaseModel, Seed, QcfLegGenerator):
    overnight_index_leg_template: OvernightIndexLegTemplate
    multi_currency_template: MultiCurrencySeed
    
    def get_parameters(
            self, 
            seed: OvernightIndexLegSeed, 
            calendars: dict[str, qcf.BusinessCalendar], 
            indices: (dict)[str, Any]
    ) -> dict[str, Any]:
        seed_parameters = seed.get_parameters()
        mccy_parameters = self.multi_currency_template.get_parameters(indices)
        template_parameters = self.overnight_index_leg_template.get_parameters(calendars)
        return seed_parameters | mccy_parameters | template_parameters
    
    def get_qcf_leg(
            self, 
            seed: OvernightIndexLegSeed, 
            calendars: dict[str, qcf.BusinessCalendar],
            indices: dict[str, Any],
    ):
        all_parameters = self.get_parameters(seed, calendars, indices)
        return qcf.LegFactory.build_bullet_overnight_index_multi_currency_leg(**all_parameters)

In [85]:
overnight_index_leg_mmcy = MultiCurrencySeed(
    settlement_currency=qcw.Currency.USD,
    fx_rate_index_name="USDCLP_OBS",
    fx_rate_index_fixing_lag=0,
)

In [87]:
overnight_index_mccy_leg_template = OvernightIndexMultiCurrencyLegTemplate(
    overnight_index_leg_template=overnight_index_leg_template,
    multi_currency_template=overnight_index_leg_mmcy,
)

In [91]:
aux.leg_as_dataframe(overnight_index_mccy_leg_template.get_qcf_leg(overnight_index_leg_seed, all_calendars, all_fx_rate_indices))

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,...,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2024-06-28,2024-12-30,2024-06-28,2024-12-30,2024-12-31,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,...,-5.138889e+06,0.01,1.0,USD,USDCLP_OBS,2024-12-31,1.0,-5138889.0,0.000000e+00,-5.138889e+06
1,2024-12-30,2025-06-30,2024-12-30,2025-06-30,2025-07-01,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,...,-5.055556e+06,0.01,1.0,USD,USDCLP_OBS,2025-07-01,1.0,-5055556.0,0.000000e+00,-5.055556e+06
2,2025-06-30,2025-12-29,2025-06-30,2025-12-29,2025-12-30,-1.000000e+09,0.000000e+00,True,CLP,ICPCLP,...,-5.055556e+06,0.01,1.0,USD,USDCLP_OBS,2025-12-30,1.0,-5055556.0,0.000000e+00,-5.055556e+06
3,2025-12-29,2026-06-29,2025-12-29,2026-06-29,2026-06-30,-1.000000e+09,-1.000000e+09,True,CLP,ICPCLP,...,-1.005056e+09,0.01,1.0,USD,USDCLP_OBS,2026-06-30,1.0,-5055556.0,-1.000000e+09,-1.005056e+09


## Product

In [117]:
AnySeed = Union[
    FixedRateLegSeed,
    OvernightIndexLegSeed,
]

AnyTemplate = Union[
    FixedRateLegTemplate,
    FixedRateMultiCurrencyLegTemplate,
    OvernightIndexLegTemplate,
    OvernightIndexMultiCurrencyLegTemplate,
]

In [121]:
class ProductSide(BaseModel):
    templates: list[AnyTemplate]
    
class Product(BaseModel):
    product_side_one: ProductSide
    product_side_two: ProductSide

In [119]:
class OperationSide(BaseModel):
    counterparty: str
    seed: AnySeed
    product_side: ProductSide
    
class Operation(BaseModel):
    description: dict[str, Any]
    side_one: OperationSide
    side_two: OperationSide